In [1]:
!pip install cassandra-driver

   ---------------------------------------- 2.9/2.9 MB 8.7 MB/s eta 0:00:00


In [2]:
from cassandra.cluster import Cluster

In [6]:
cluster = Cluster(['192.168.15.3'], port=9042)
session = cluster.connect('instructor')

instructor = session.execute('select * from instructor;')
for i in instructor:
    print(i)

NoHostAvailable: ('Unable to connect to any servers', {'0.0.0.0:9042': OSError(10049, "Tried connecting to [('0.0.0.0', 9042)]. Last error: O endereço solicitado não é válido no contexto")})